In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

import warnings
warnings.filterwarnings('ignore', category = DeprecationWarning)

Using TensorFlow backend.


In [2]:
# dimensions of our images.

img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

nb_train_samples = 2000
nb_validation_samples = 800

epochs = 10
batch_size = 16

In [3]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

W0818 10:44:25.199241  9336 deprecation_wrapper.py:119] From C:\Users\atuls\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0818 10:44:25.218584  9336 deprecation_wrapper.py:119] From C:\Users\atuls\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0818 10:44:25.221809  9336 deprecation_wrapper.py:119] From C:\Users\atuls\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4420: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0818 10:44:25.239809  9336 deprecation_wrapper.py:119] From C:\Users\atuls\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4255: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0818 10:44:25.316704  9336 deprecation_wrapper.py:119] From C:\Users\atuls\Ana

In [5]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

W0818 10:44:25.403681  9336 deprecation_wrapper.py:119] From C:\Users\atuls\Anaconda3\lib\site-packages\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0818 10:44:25.439220  9336 deprecation_wrapper.py:119] From C:\Users\atuls\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3645: The name tf.log is deprecated. Please use tf.math.log instead.

W0818 10:44:25.444870  9336 deprecation.py:323] From C:\Users\atuls\Anaconda3\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [9]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

# save model and architecture to single file
model.save("full_model.h5")
print("Saved full model to disk")

Epoch 1/10
125/125 [==============================] - 56s 450ms/step - loss: 0.7030 - acc: 0.5290 - val_loss: 0.6786 - val_acc: 0.5038
Epoch 2/10
125/125 [==============================] - 56s 445ms/step - loss: 0.6908 - acc: 0.6030 - val_loss: 0.6637 - val_acc: 0.5900
Epoch 3/10
125/125 [==============================] - 55s 439ms/step - loss: 0.6397 - acc: 0.6420 - val_loss: 0.6235 - val_acc: 0.6425
Epoch 4/10
125/125 [==============================] - 57s 454ms/step - loss: 0.6231 - acc: 0.6625 - val_loss: 0.6328 - val_acc: 0.6338
Epoch 5/10
125/125 [==============================] - 55s 440ms/step - loss: 0.6098 - acc: 0.6655 - val_loss: 0.5791 - val_acc: 0.6937
Epoch 6/10
125/125 [==============================] - 57s 455ms/step - loss: 0.5978 - acc: 0.6785 - val_loss: 0.6191 - val_acc: 0.6350
Epoch 7/10
125/125 [==============================] - 56s 447ms/step - loss: 0.5780 - acc: 0.6990 - val_loss: 0.7884 - val_acc: 0.5975
Epoch 8/10
125/125 [==============================] - 5

In [10]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [11]:
import pprint
pprint.pprint(model_json)

('{"class_name": "Sequential", "config": {"name": "sequential_1", "layers": '
 '[{"class_name": "Conv2D", "config": {"name": "conv2d_1", "trainable": true, '
 '"batch_input_shape": [null, 150, 150, 3], "dtype": "float32", "filters": 32, '
 '"kernel_size": [3, 3], "strides": [1, 1], "padding": "valid", "data_format": '
 '"channels_last", "dilation_rate": [1, 1], "activation": "linear", '
 '"use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", '
 '"config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", '
 '"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, '
 '"kernel_regularizer": null, "bias_regularizer": null, '
 '"activity_regularizer": null, "kernel_constraint": null, "bias_constraint": '
 'null}}, {"class_name": "Activation", "config": {"name": "activation_1", '
 '"trainable": true, "dtype": "float32", "activation": "relu"}}, '
 '{"class_name": "MaxPooling2D", "config": {"name": "max_pooling2d_1", '
 '"trainable": true, "dt

In [13]:
from keras.models import load_model
 
# load model
full_model = load_model('full_model.h5')
# summarize model.
full_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)       